In [1]:
# Install dependencies
!pip install  bitsandbytes accelerate gradio openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s e

In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import gradio as gr
import whisper
import functools

# ✅ Step 1: Load Falcon-7B-Instruct with 4-bit Quantization
model_name = "tiiuae/falcon-7b-instruct"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map=device
)

print("✅ Falcon-7B-Instruct Loaded Successfully on:", device)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

✅ Falcon-7B-Instruct Loaded Successfully on: cuda


In [1]:
import json

# ✅ Define the training prompts
training_data = {
    "storytelling": [
        "Tell a story about a challenge you overcame.",
        "Describe a time when you helped someone in need.",
        "Imagine a futuristic world and narrate a short story.",
        "Tell a story about a childhood memory that shaped you.",
        "Share an experience where you had to make a difficult choice.",
        "Narrate a funny or embarrassing moment and what you learned from it.",
        "Describe a time when a stranger impacted your life in an unexpected way.",
        "Tell a story about a travel experience that changed your perspective.",
        "Recount a time when you faced failure and how it helped you grow.",
        "Share a moment when you discovered something new about yourself."
    ],
    "impromptu_speaking": [
        "If you could invent a new technology, what would it be?",
        "Describe a moment that changed your life.",
        "What advice would you give to your younger self?",
        "If you could live in any historical period, which one and why?",
        "If you could have dinner with any historical figure, who would it be and why?",
        "What is one thing you would change about the world and why?",
        "If you won a million dollars today, what would you do with it?",
        "What is the most important skill for success in the modern world?",
        "Describe a book or movie that had a significant impact on you.",
        "If you had to teach a class on any subject, what would it be and why?"
    ],
    "conflict_resolution": [
        "A coworker takes credit for your work. How do you handle it?",
        "You have a disagreement with your manager. How do you resolve it?",
        "A team member is not contributing to a group project. What do you do?",
        "You need to deliver bad news to a client. How do you approach it?",
        "A customer is unhappy with your service. How do you handle the situation?",
        "You have a conflict with a close friend over a misunderstanding. How do you fix it?",
        "Two team members are in a heated argument during a meeting. How do you mediate?",
        "You are negotiating a deal, but the other party is being unreasonable. What is your approach?",
        "A family member disagrees with your life choices. How do you communicate effectively?",
        "A colleague keeps interrupting you during meetings. How do you address it professionally?"
    ]
}

# ✅ Save it to a JSON file
with open("training_prompts.json", "w") as f:
    json.dump(training_data, f, indent=4)

print("✅ training_prompts.json file updated successfully!")


✅ training_prompts.json file updated successfully!


In [4]:
import os

def load_training_prompts():
    """Load predefined training prompts from JSON."""
    if not os.path.exists("training_prompts.json"):
        print("⚠️ training_prompts.json not found! Creating a default file...")
        # Auto-create the file if missing
        with open("training_prompts.json", "w") as f:
            json.dump(training_data, f)

    with open("training_prompts.json", "r") as f:
        return json.load(f)

# ✅ Load the training data
training_data = load_training_prompts()


In [5]:
# ✅ Load Whisper Model for Speech-to-Text
whisper_model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:08<00:00, 54.7MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [71]:
# ✅ AI Functions
chat_history = []

def chat_with_ai(input_type, user_input, audio_input):
    """Handles user choice for text or voice input and returns only the last AI response"""
    if input_type == "Voice":
        if audio_input is None:
            return "❌ No audio file provided. Please record your voice."
        user_input = transcribe_audio(audio_input)  # Convert speech to text

    chat_history.append(f"User: {user_input}")

    # Limit history to last 10 exchanges to prevent excessive context length
    if len(chat_history) > 10:
        chat_history.pop(0)

    prompt = f"User: {user_input}\nAssistant:"

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_new_tokens=200, temperature=0.9, top_p=0.95)

    response = tokenizer.decode(output[0], skip_special_tokens=True)

    chat_history.append(f"Assistant: {response}")  # Store AI response in history

    return response  # Only returning the latest response, no full history


In [24]:
def load_training_prompts():
    """Load predefined training prompts from JSON."""
    with open("training_prompts.json", "r") as f:
        return json.load(f)

training_data = load_training_prompts()

In [63]:
import json
import random

def storytelling_prompt():
    """Pick a random storytelling prompt."""
    return random.choice(training_data["storytelling"])

def evaluate_story(user_text):
    """Evaluate storytelling quality with scoring and force critical feedback."""

    if not user_text or len(user_text.split()) < 5:
        return "❌ This input is too short to be a story. Please provide a complete story."

    prompt = f"""
You are a **professional storytelling coach and literary critic**. Your task is to analyze and critique the following story with a **critical eye**.
**If the input is meaningless, incomplete, or gibberish, call it out immediately!**
If the story lacks structure or creativity, **point it out clearly and explain why.**

### **📌 Evaluation Criteria (Score 1-10)**
1️⃣ **Story Structure:**
   - Does the story have a clear **beginning, middle, and end**?
   - Are transitions smooth, or does it feel disjointed?

2️⃣ **Emotional Engagement:**
   - Does the story make the reader **feel** something? (Joy, sadness, excitement, suspense)
   - Are the characters and events **relatable & immersive**?

3️⃣ **Creativity & Originality:**
   - Does it introduce **unique ideas or perspectives**?
   - Does it avoid clichés and predictable storylines?

4️⃣ **Clarity & Flow:**
   - Is the writing **coherent and easy to follow**?
   - Is the pacing **appropriate** (not too rushed or too slow)?

### **🔥 Scoring Format:**
- **Story Structure:** X/10
- **Emotional Engagement:** X/10
- **Creativity & Originality:** X/10
- **Clarity & Flow:** X/10

---

### **⚠️ Important: Call Out Bad Writing!**
- If the story is **gibberish, unstructured, or makes no sense**, **say so directly.**
- If it is **too generic**, **explain why it lacks originality.**
- Do **NOT** give high scores unless the story genuinely deserves them.

---

### **💡 Actionable Improvement Tips:**
✅ Provide **3-5 specific, non-generic ways** to make the story better.

---

📖 **Story to Evaluate:**
{user_text}

🎯 **Now, provide your honest, detailed critique.**
"""

    response = generate_response(prompt)
    return response.split("Assistant:")[-1].strip()


In [62]:
def generate_response(user_input):
    """Generates AI response with optimized parameters for speed and accuracy."""

    formatted_input = f"User: {user_input}\nAssistant:"
    inputs = tokenizer(formatted_input, return_tensors="pt").to(device)

    # Generate output with optimized settings
    output = model.generate(
        **inputs,
        max_new_tokens=200,  # Reduced token limit for faster processing
        temperature=0.9,  # More balanced randomness
        top_p=0.9  # More controlled probability selection
    )

    # Decode response
    response = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # **Fix: Remove unwanted prompt repetition**
    if "Assistant:" in response:
        response = response.split("Assistant:")[-1].strip()

    return response


In [61]:
import re
import random


def impromptu_speaking():
    """Pick a random topic for impromptu speaking."""
    return random.choice(training_data["impromptu_speaking"])

def evaluate_speech(user_text):
    """Evaluate impromptu speech with critical feedback and topic relevance checking."""

    if not user_text or len(user_text.split()) < 5:
        return "❌ This response is too short to be a valid speech. Please provide a complete response."


    # **Step 2: Detailed Speech Evaluation**
    prompt = f"""
You are a **professional speech coach and public speaking expert**.
Your job is to analyze and **critique this speech thoroughly** with a **critical eye**.
**If the speech lacks relevance, structure, or confidence, point it out clearly.**

### **📌 Evaluation Criteria (Score 1-10 per category)**
1️⃣ **Relevance to Topic:**
   - Does the speech **directly** address the given topic?
   - Is the content **on-topic** or does it drift off?

2️⃣ **Clarity & Structure:**
   - Does it have a **clear introduction, body, and conclusion**?
   - Are transitions **smooth and logical**, or does it feel disjointed?

3️⃣ **Engagement & Persuasion:**
   - Does the speaker **capture the audience’s attention**?
   - Are rhetorical devices (stories, examples, humor) used effectively?

4️⃣ **Pacing & Confidence:**
   - Does the speaker sound **confident and natural**?
   - Is the **pacing appropriate** (not too rushed or too slow)?
   - Are there **too many filler words** ("um", "uh", "you know")?

### **🔥 Scoring Format:**
- **Relevance to Topic:** X/10
- **Clarity & Structure:** X/10
- **Engagement & Persuasion:** X/10
- **Pacing & Confidence:** X/10

---

### **⚠️ Important: Call Out Weak Responses!**
- If the response is **off-topic**, **say so directly.**
- If it is **unclear or lacks depth**, **explain why and how to fix it.**
- **Do NOT give high scores unless truly deserved.**

---

### **💡 Actionable Improvement Tips:**
✅ Provide **3-5 specific, non-generic ways** to improve the speech.

---

🎙 **User’s Speech:** {user_text}

🎯 **Now, provide your honest, detailed critique.**
"""

    response = generate_response(prompt)
    return response.strip()


In [60]:
def conflict_resolution_scenario():
    """Pick a random conflict resolution scenario."""
    return random.choice(training_data["conflict_resolution"])

def assess_conflict_response(user_text):
    """Critically analyze conflict resolution response and ensure honest, structured feedback."""

    if not user_text or len(user_text.split()) < 5:
        return "❌ This input is too short to assess. Please provide a full response."

    prompt = f"""
You are a **professional conflict resolution coach and negotiation expert**.
Your task is to **analyze this response critically** and **highlight both strengths & flaws**.
Do **NOT** be overly positive—**if the response is weak, say so directly!**

### **📌 Evaluation Criteria (Score 1-10 per category)**
1️⃣ **Empathy & Understanding:**
   - Does the response show genuine **concern for others' emotions**?
   - Does it acknowledge the other person's **perspective & feelings**?

2️⃣ **Diplomacy & Professionalism:**
   - Is the response **calm, professional, and respectful**?
   - Does it avoid **aggression, defensiveness, or passive-aggressiveness**?

3️⃣ **Effectiveness of Resolution:**
   - Does the response **actually solve the conflict**?
   - Does it suggest a **reasonable compromise or constructive solution**?

### **🔥 Scoring Format:**
- **Empathy & Understanding:** X/10
- **Diplomacy & Professionalism:** X/10
- **Effectiveness of Resolution:** X/10

---

### **⚠️ Important: Call Out Weak Responses!**
❌ If the response lacks empathy, **explain why**.
❌ If it's aggressive, vague, or dismissive, **highlight it clearly**.
❌ If it doesn't resolve the conflict, **suggest a better approach**.

---

### **💡 Actionable Improvement Tips:**
✅ Provide **3-5 clear, specific ways** to improve the response.
✅ Avoid generic feedback—**make it personal & precise**.

---

🤝 **User’s Conflict Resolution Response:**
{user_text}

🎯 **Now, provide your honest, structured critique.**
"""

    response = generate_response(prompt)
    return response.split("Assistant:")[-1].strip()


In [50]:
def transcribe_audio(audio_file):
    """Convert speech to text"""
    result = whisper_model.transcribe(audio_file)
    return result["text"]

In [64]:
def assess_presentation(user_text):
    """AI strictly analyzes speech and provides HONEST, CRITICAL feedback with actual scores."""

    if not user_text:
        return "❌ Please enter or record a speech before submitting."

        prompt = f"""
You are a **professional speech coach and public speaking expert**.
Your job is to analyze and **critique this speech thoroughly** with a **critical eye**.
**If the speech lacks relevance, structure, or confidence, point it out clearly.**

### **📌 Evaluation Criteria (Score 1-10 per category)**
1️⃣ **Relevance to Topic:**
   - Does the speech **directly** address the given topic?
   - Is the content **on-topic** or does it drift off?

2️⃣ **Clarity & Structure:**
   - Does it have a **clear introduction, body, and conclusion**?
   - Are transitions **smooth and logical**, or does it feel disjointed?

3️⃣ **Engagement & Persuasion:**
   - Does the speaker **capture the audience’s attention**?
   - Are rhetorical devices (stories, examples, humor) used effectively?

4️⃣ **Pacing & Confidence:**
   - Does the speaker sound **confident and natural**?
   - Is the **pacing appropriate** (not too rushed or too slow)?
   - Are there **too many filler words** ("um", "uh", "you know")?

### **🔥 Scoring Format:**
- **Relevance to Topic:** X/10
- **Clarity & Structure:** X/10
- **Engagement & Persuasion:** X/10
- **Pacing & Confidence:** X/10

---

### **⚠️ Important: Call Out Weak Responses!**
- If the response is **off-topic**, **say so directly.**
- If it is **unclear or lacks depth**, **explain why and how to fix it.**
- **Do NOT give high scores unless truly deserved.**

---

### **💡 Actionable Improvement Tips:**
✅ Provide **3-5 specific, non-generic ways** to improve the speech.

---

🎙 **User’s Speech:** {user_text}

🎯 **Now, provide your honest, detailed critique.**
"""

    response = generate_response(prompt)
    return response.strip()



In [65]:
def analyze_speech_tone(user_text):
    """Critically analyzes speech tone and ensures structured, professional feedback."""

    if not user_text or len(user_text.split()) < 5:
        return "❌ This input is too short to assess. Please provide a full speech."

    prompt = f"""
You are a **professional speech coach and vocal expert**.
Your job is to **analyze and critique this speech tone thoroughly** with a **critical eye**.
**If the speech lacks confidence, vocal variety, or expressiveness, point it out clearly.**

### **📌 Evaluation Criteria (Score 1-10 per category)**
1️⃣ **Confidence & Presence:**
   - Does the speaker **sound self-assured or hesitant**?
   - Do they convey **authority and credibility**?

2️⃣ **Emotional Tone & Expressiveness:**
   - Is the speech **engaging, passionate, and expressive**?
   - Does it sound **robotic, flat, or overly dramatic**?

3️⃣ **Intonation & Vocal Variability:**
   - Does the speaker **change pitch naturally** or stay monotone?
   - Are key points **emphasized effectively**?

4️⃣ **Filler Words & Hesitation:**
   - Does the speaker **use too many "um", "uh", or long pauses**?
   - Do they hesitate or struggle to find words?

### **🔥 Scoring Format:**
- **Confidence & Presence:** X/10
- **Emotional Tone & Expressiveness:** X/10
- **Intonation & Vocal Variability:** X/10
- **Filler Words & Hesitation:** X/10

---

### **⚠️ Important: Call Out Weak Speech Tones!**
❌ If the speech is **monotone, hesitant, or lacks expressiveness**, **highlight the issue clearly**.
❌ If there’s **no vocal variation**, AI must call it out as monotonous.
❌ If **filler words** are excessive, AI must explain their impact on the speech.

---

### **💡 Actionable Improvement Tips:**
✅ Provide **3-5 specific, non-generic ways** to improve tone.
✅ Avoid vague advice—**make it precise and tailored to the speech**.

---

🎤 **User’s Speech:**
{user_text}

🎯 **Now, provide your structured, detailed critique.**
"""


    response = generate_response(prompt)
    return response.split("Assistant:")[-1].strip()

In [66]:
import os

def save_feedback(user_text, ai_feedback):
    """Saves AI feedback into a properly formatted JSON file for progress tracking."""

    file_path = "user_progress.json"

    # Load existing data if the file exists, else start a new list
    if os.path.exists(file_path):
        try:
            with open(file_path, "r") as f:
                data = json.load(f)  # Load existing JSON data
        except json.JSONDecodeError:
            data = []  # If file is corrupted, reset it
    else:
        data = []  # Create a new list if file does not exist

    # Append new entry
    data.append({"speech": user_text, "feedback": ai_feedback})

    # Save back to file in a structured format
    with open(file_path, "w") as f:
        json.dump(data, f, indent=4)  # Pretty-print JSON with indentation

    return "✅ Feedback successfully saved!"



In [67]:

from datetime import datetime

# ✅ File to store user progress

PROGRESS_FILE = "progress.json"

def save_feedback(user_text, ai_feedback, topic, scores):
    """Saves AI feedback, topic, and scores into a JSON file for tracking progress"""
    data = {
        "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "topic": topic,
        "speech": user_text,
        "scores": scores,  # Dictionary of individual category scores
        "feedback": ai_feedback
    }

    # Load existing progress or create a new list
    try:
        with open(PROGRESS_FILE, "r") as f:
            progress_data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        progress_data = []

    # Append new data
    progress_data.append(data)

    # Save back to file
    with open(PROGRESS_FILE, "w") as f:
        json.dump(progress_data, f, indent=4)

    return "✅ Feedback & scores saved for progress tracking!"


In [68]:
def load_progress():
    """Loads user progress from JSON and returns formatted results."""
    try:
        with open(PROGRESS_FILE, "r") as f:
            progress_data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return "⚠️ No progress data found."

    if not progress_data:
        return "⚠️ No past results available yet."

    progress_summary = "📊 **Your Past Performance:**\n\n"

    for entry in progress_data[-5:]:  # Show last 5 results
        progress_summary += f"📅 **Date:** {entry['date']}\n"
        progress_summary += f"📝 **Topic:** {entry['topic']}\n"
        progress_summary += "🎯 **Scores:**\n"
        for category, score in entry["scores"].items():
            progress_summary += f"   - {category}: {score}/10\n"
        progress_summary += f"💡 **Feedback:** {entry['feedback'][:200]}...\n\n"  # Show a preview

    return progress_summary


In [69]:
def get_best_scores():
    """Finds the highest score recorded for each evaluation category."""
    try:
        with open(PROGRESS_FILE, "r") as f:
            progress_data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return "⚠️ No progress data available."

    if not progress_data:
        return "⚠️ No past results available."

    best_scores = {}

    for entry in progress_data:
        for category, score in entry["scores"].items():
            if category not in best_scores or score > best_scores[category]:
                best_scores[category] = score

    best_scores_text = "🏆 **Your Best Scores:**\n"
    for category, score in best_scores.items():
        best_scores_text += f"✅ {category}: {score}/10\n"

    return best_scores_text


In [72]:
# ✅ Build Unified Gradio UI with All Features

import gradio as gr

# Custom CSS for a premium look
custom_css = """
/* 🎨 General Styling */
h1, h2 {
    text-align: center;
    font-family: 'Poppins', sans-serif;
    font-size: 2.2em;
    color: #ffffff;
    font-weight: bold;
}

p {
    text-align: center;
    font-family: 'Poppins', sans-serif;
    font-size: 1.2em;
    color: #ffffff;
}

/* 🌌 Background */
body {
    background: linear-gradient(to right, #1e3c72, #2a5298); /* Deep Blue Theme */
    font-family: 'Poppins', sans-serif;
    margin: 0;
    padding: 0;
}

/* 🔹 Fix Tab Title Colors */
button[role="tab"] {
    color: white !important;
    font-weight: bold;
    background: rgba(255, 255, 255, 0.1); /* Soft Transparent Background */
    border-radius: 10px;
    transition: 0.3s ease-in-out;
}

button[role="tab"]:hover {
    background: rgba(255, 255, 255, 0.2);
}

/* 🚀 Improve Container Visibility */
.gradio-container {
    max-width: 900px;
    margin: auto;
    padding: 25px;
    background: rgba(0, 0, 0, 0.85);
    border-radius: 15px;
    box-shadow: 0px 0px 25px rgba(255, 255, 255, 0.2);
}

/* 🔘 Button Styling */
.gr-button {
    background: linear-gradient(90deg, #ff7e5f, #feb47b); /* Orange-Red Gradient */
    color: white !important;
    font-weight: bold;
    border-radius: 10px;
    padding: 12px 25px !important;
    border: none;
    transition: 0.3s ease-in-out;
}

.gr-button:hover {
    background: linear-gradient(90deg, #ff5e62, #ff9966); /* Darker Gradient */
    transform: scale(1.05);
}

/* 📢 Improve Input Fields */
.gr-textbox, .gr-textarea, .gr-audio {
    border-radius: 8px;
    background: rgba(255, 255, 255, 0.1);
    border: 1px solid rgba(255, 255, 255, 0.3);
    color: white;
    padding: 10px;
    transition: 0.3s ease-in-out;
}

.gr-textbox:focus, .gr-textarea:focus {
    border: 1px solid #ff9966;
    background: rgba(255, 255, 255, 0.15);
}

/* 🔉 Audio & File Upload Enhancements */
.gr-audio {
    display: flex;
    justify-content: center;
    align-items: center;
    width: 100%;
    padding: 10px;
    background: rgba(255, 255, 255, 0.1);
    border-radius: 8px;
    border: 1px solid rgba(255, 255, 255, 0.3);
}

/* 📁 File Upload Box */
.gr-file {
    background: rgba(255, 255, 255, 0.1);
    border: 1px dashed rgba(255, 255, 255, 0.3);
    padding: 15px;
    text-align: center;
    border-radius: 8px;
    transition: 0.3s ease-in-out;
}

.gr-file:hover {
    border-color: #ff9966;
    background: rgba(255, 255, 255, 0.15);
}

/* 🏆 Better Tab Navigation */
.gradio-tabs {
    display: flex;
    justify-content: center;
    margin-bottom: 15px;
}

button[role="tab"][aria-selected="true"] {
    background: #ff9966 !important;
    color: white !important;
}

"""

with gr.Blocks(css=custom_css) as interface:
    gr.Markdown("# 🎤 AI Verbal Communication Trainer")
    gr.Markdown("Improve your speaking skills with real-time AI feedback! 🚀")

    # Main Priority Tabs

    with gr.Tab("Training Activities"):
        with gr.Accordion("🗣️ Impromptu Speaking",open=False):
            gr.Markdown("## 🎙️ Impromptu Speaking Practice")
            impromptu_button = gr.Button("🎲 Generate Topic", variant="secondary")
            impromptu_topic = gr.Textbox(label="Your Topic", interactive=False)

            input_type = gr.Radio(["Text", "Voice"], label="Input Type", value="Text")
            impromptu_response = gr.Textbox(label="Enter your speech", placeholder="Type or record your speech...")
            impromptu_audio = gr.Audio(type="filepath", label="🎙️ Record your speech")
            impromptu_eval_button = gr.Button("📊 Evaluate Speech", variant="primary")
            impromptu_feedback = gr.Textbox(label="AI Feedback", interactive=False)

            impromptu_button.click(impromptu_speaking, outputs=impromptu_topic)
            impromptu_eval_button.click(
                lambda t, a: evaluate_speech(transcribe_audio(a) if t == "Voice" else t),
                inputs=[impromptu_response, impromptu_audio],
                outputs=impromptu_feedback
            )

        with gr.Accordion("📖 Storytelling",open=False):
            gr.Markdown("## 📚 Storytelling Practice")
            story_button = gr.Button("📖 Generate Story Prompt", variant="secondary")
            story_topic = gr.Textbox(label="Your Story Topic", interactive=False)

            input_type = gr.Radio(["Text", "Voice"], label="Input Type", value="Text")
            story_response = gr.Textbox(label="Enter your story", placeholder="Type or record your story...")
            story_audio = gr.Audio(type="filepath", label="🎙️ Record your story")
            story_eval_button = gr.Button("📊 Evaluate Story", variant="primary")
            story_feedback = gr.Textbox(label="AI Feedback", interactive=False)

            story_button.click(storytelling_prompt, outputs=story_topic)
            story_eval_button.click(
                lambda t, a: evaluate_story(transcribe_audio(a) if t == "Voice" else t),
                inputs=[story_response, story_audio],
                outputs=story_feedback
            )

        with gr.Accordion("🤝 Conflict Resolution",open=False):
            gr.Markdown("## 🔥 Conflict Resolution Training")
            conflict_button = gr.Button("📝 Generate Scenario", variant="secondary")
            conflict_topic = gr.Textbox(label="Your Scenario", interactive=False)

            input_type = gr.Radio(["Text", "Voice"], label="Input Type", value="Text")
            conflict_response = gr.Textbox(label="Enter your response", placeholder="Type or record your response...")
            conflict_audio = gr.Audio(type="filepath", label="🎙️ Record your response")
            conflict_eval_button = gr.Button("📊 Evaluate Response", variant="primary")
            conflict_feedback = gr.Textbox(label="AI Feedback", interactive=False)

            conflict_button.click(conflict_resolution_scenario, outputs=conflict_topic)
            conflict_eval_button.click(
                lambda t, a: assess_conflict_response(transcribe_audio(a) if t == "Voice" else t),
                inputs=[conflict_response, conflict_audio],
                outputs=conflict_feedback
            )




    # Less Priority Tabs (Inside "More" Menu)
    with gr.Tab("Extra activities"):
        with gr.Accordion("💬 Chat with AI", open=False):
            gr.Markdown("## 🗣️ Speak with AI")
            input_type = gr.Radio(["Text", "Voice"], label="Input Type", value="Text")
            text_input = gr.Textbox(label="Enter text here", placeholder="Type your message...")
            audio_input = gr.Audio(type="filepath", label="🎙️ Record your voice")
            submit_button = gr.Button("💬 Chat", variant="primary")
            output_box = gr.Textbox(label="Response", interactive=False)

            submit_button.click(chat_with_ai, inputs=[input_type, text_input, audio_input], outputs=output_box)

        with gr.Accordion("📢 Speech Analysis", open=False):
            gr.Markdown("## 🎤 AI Speech Feedback")
            input_type = gr.Radio(["Text", "Voice"], label="Input Type", value="Text")
            speech_input = gr.Textbox(label="Paste your speech", placeholder="Type or record your speech...")
            speech_audio = gr.Audio(type="filepath", label="🎙️ Record your speech")
            analyze_button = gr.Button("📊 Analyze Speech", variant="primary")
            speech_output = gr.Textbox(label="AI Feedback", interactive=False)

            analyze_button.click(
                lambda t, a: assess_presentation(transcribe_audio(a) if t == "Voice" else t),
                inputs=[speech_input, speech_audio],
                outputs=speech_output
            )

        with gr.Accordion("🎭 Speech Tone Analysis", open=False):
            gr.Markdown("## 🎭 Speech Tone Detection")
            input_type = gr.Radio(["Text", "Voice"], label="Input Type", value="Text")
            tone_input = gr.Textbox(label="Paste your speech", placeholder="Type or record your speech...")
            tone_audio = gr.Audio(type="filepath", label="🎙️ Record your voice")
            tone_button = gr.Button("📊 Analyze Tone", variant="primary")
            tone_output = gr.Textbox(label="AI Tone Analysis", interactive=False)

            tone_button.click(
                lambda t, a: analyze_speech_tone(transcribe_audio(a) if t == "Voice" else t),
                inputs=[tone_input, tone_audio],
                outputs=tone_output
            )
    with gr.Tab("📈 View Progress"):
            gr.Markdown("## 📊 Your Progress & Best Scores")

            progress_button = gr.Button("📂 Load My Progress")
            progress_output = gr.Textbox(label="Your Past Performance", interactive=False)

            best_score_button = gr.Button("🏆 Show Best Scores")
            best_score_output = gr.Textbox(label="Best Scores", interactive=False)

            progress_button.click(load_progress, outputs=progress_output)
            best_score_button.click(get_best_scores, outputs=best_score_output)

interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://840300e46cd620a35c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
